1. Introduction
1. Data preparation
    1. Load data
    1. Normalization, Reshape and Label encoding
    1. Visualize test and train sample
1. Model Building
    1. Split training and valdiation set
    1. Define the model architechture
    1. Set the optimizer and annealer
    1. Data augmentation
    1. Train model
1. Evaluate the model
    1. Training and validation curves
    1. Visualize Prediction
1. Prediction and submition
    1. Predict and Submit results
* * * *
### 1. Introduction

This kernel is basic start in deep learning. 

CIFAR-10 (Canadian Institute For Advanced Research) is the type “hello world” dataset of computer vision. This dataset is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes. The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class. In this competition, your goal is to correctly identify different object from a dataset of tens of thousands of color images.
* * * *

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import backend
import tensorflow as  tf

# Model architecture
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D
from keras.layers import MaxPool2D, Activation, MaxPooling2D
from keras.layers.normalization import BatchNormalization


# model optimisation and scores
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, auc, roc_curve


# Annealer
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Data processing
# from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
# from keras.preprocessing import image

# Visualization
import matplotlib.pyplot as plt

import h5py


**File Structure:**

* **train.7z -** a folder containing the training images
* **test.7z -** a folder containing the test images
* **trainLabels.csv -** the training labels
* **sampleSubmission.csv -** We have to predict labels for all 300,000 images.

Train and Test files are present a .7z compressed format. You can process file from .7z extension; it will take long enough time to process.
<pre><code>
# Reading .7z content
!pip install pyunpack # install decoder package
!pip install patool # install requirement

from pyunpack import Archive

print("start unpacking train data")
%time Archive('/kaggle/input/cifar-10/train.7z').extractall('/kaggle/working/dataset/', auto_create_dir=True, patool_path=None)
print("unpacking finished")
print("start unpacking test data")
%time Archive('/kaggle/input/cifar-10/test.7z').extractall('/kaggle/working/dataset/', auto_create_dir=True, patool_path=None)
print("unpacking finished")

### This will take approx 2-3 hr to read file from test.7z.

</code></pre>

I downloaded the zip file, and processed it and created files in batches.You can use this data for your work, data is organised as follow:
* **train_data.h5-** it is generated from train.7z, contain training samples.
* **test_data_1.h5-** first 50k examples from test.7z
* **test_data_2.h5-** index 50k to 100k examples from test.7z
* **test_data_3.h5- ** index 100k to 150k examples from test.7z
* **test_data_4.h5-** index 150k to 200k examples from test.7z
* **test_data_5.h5- ** index 200k to 250k examples from test.7z
* **test_data_6.h5-** index 250k to 300k examples from test.7z


### 2. Data Prepration
**Load Data**

In [ ]:
# Read File
sample_submission = pd.read_csv('../input/cifar-10/sampleSubmission.csv')
train_labels = pd.read_csv('../input/cifar-10/trainLabels.csv')

print("Number of training sample: ",train_labels.shape[0])
print("Number of test samples: ", sample_submission.shape[0])

In [ ]:
sample_submission

In [ ]:
import h5py
with h5py.File('../input/cifardata/train_data.h5', 'r') as file:
    #for key in file.keys():
     #   print(key)
    train_ids = pd.DataFrame(np.array(np.squeeze(file['train_ids'])),columns=['id'])
    train_data = np.array(file['train_images']).reshape(-1, 32, 32, 3)
    
with h5py.File('../input/cifardata/test_data_1.h5', 'r') as file:
    test_ids_1 = pd.DataFrame(np.array(np.squeeze(file['test_ids'])),columns=['id'])
    test_data_1 = np.array(file['test_images']).reshape(-1, 32, 32, 3)
    
with h5py.File('../input/cifardata/test_data_2.h5', 'r') as file:
    test_ids_2 = pd.DataFrame(np.array(np.squeeze(file['test_ids'])),columns=['id'])
    test_data_2 = np.array(file['test_images']).reshape(-1, 32, 32, 3)
    
    
with h5py.File('../input/cifardata/test_data_3.h5', 'r') as file:
    test_ids_3 = pd.DataFrame(np.array(np.squeeze(file['test_ids'])),columns=['id'])
    test_data_3 = np.array(file['test_images']).reshape(-1, 32, 32, 3)
    
    
with h5py.File('../input/cifardata/test_data_4.h5', 'r') as file:
    test_ids_4 = pd.DataFrame(np.array(np.squeeze(file['test_ids'])),columns=['id'])
    test_data_4 = np.array(file['test_images']).reshape(-1, 32, 32, 3)
    
    
with h5py.File('../input/cifardata/test_data_5.h5', 'r') as file:
    test_ids_5 = pd.DataFrame(np.array(np.squeeze(file['test_ids'])),columns=['id'])
    test_data_5 = np.array(file['test_images']).reshape(-1, 32, 32, 3)
    
    
with h5py.File('../input/cifardata/test_data_6.h5', 'r') as file:
    test_ids_6 = pd.DataFrame(np.array(np.squeeze(file['test_ids'])),columns=['id'])
    test_data_6 = np.array(file['test_images']).reshape(-1, 32, 32, 3)
    


In [ ]:
# Concatenate test into one dataframe
test_ids = pd.concat([test_ids_1, test_ids_2, test_ids_3, test_ids_4, test_ids_5, test_ids_6], axis=0)
test_data = np.concatenate([test_data_1, test_data_2, test_data_3, test_data_4, test_data_5, test_data_6], axis=0)
# test_data = test_data_1
# test_ids = test_ids_1

train_data = train_data
target_variable = train_labels.label


After building dataset, let's check data is consistent or not.

In [ ]:
# checking loaded test data is consistent or not.

print("all test id and samples are consistent = ", np.all(test_ids.id.value_counts().sort_index() == sample_submission.id.value_counts().sort_index()))
print("all train ids and train_labels are consistent = ", np.all(train_ids.id == train_labels.id))

### Class Distribution

In [ ]:
distribution = train_labels.label.value_counts()
for category, size in zip(distribution.index, distribution.values):
    print(f"{category} {size} images")

In [ ]:
# Plot Distribution

plt.figure(figsize=(10, 5))
train_labels["label"].value_counts().plot(kind='bar',
                                          title='Distribution of classes'
                                         )

### Visualize Samples

In [ ]:
# function to draw samples

def draw_sample_images(data, labels = False):
    nrows = 4
    ncols = 10
    
    total_image = data.shape[0]
    

    samples = np.random.choice(total_image, nrows*ncols)

    plt.figure(figsize=(20, 5))
    for i in range(nrows*ncols):
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(data[samples[i]])
        if(labels):
            plt.title(train_labels.label[samples[i]])
        plt.axis("off")
    plt.tight_layout()
    plt.show()
            

draw_sample_images(train_data, labels = True)

### Split dataset

In [ ]:
# Label encoding
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

target_label = lb_make.fit_transform(target_variable)
target_label

In [ ]:
y = pd.get_dummies(target_label).values

In [ ]:
# split training and validation set.
X_train, X_val, Y_train, Y_val = train_test_split(train_data, y, random_state=41, test_size=0.25)

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_val shape: ", X_val.shape)
print("Y_val shape: ", Y_val.shape)

### Base Model

In [ ]:
def create_model():
        """
        Build CNN model and Perform the following operations:

        1. Flatten the output of our base model to 1 dimension
        2. Add a fully connected layer with 1,024 hidden units and ReLU activation
        3. This time, we will go with a dropout rate of 0.2
        4. Add a final Fully Connected Sigmoid Layer
        """
        
        
        model = Sequential()
        model.add(Conv2D(32,  kernel_size = 3,kernel_initializer='he_normal', activation='relu', input_shape = (32, 32, 3)))
        model.add(BatchNormalization())
        
        model.add(Dropout(0.2))
        
        model.add(Conv2D(64, kernel_size = 3, kernel_initializer='he_normal', strides=1, activation='relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, kernel_size = 3, strides=1, kernel_initializer='he_normal' ,padding='same', activation='relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, kernel_size = 3,kernel_initializer='he_normal', activation='relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling2D((4, 4)))
        model.add(Dropout(0.2))


        model.add(Flatten())
        model.add(Dense(256,kernel_initializer='he_normal', activation = "relu"))
        model.add(Dropout(0.1))
        model.add(Dense(10, kernel_initializer='glorot_uniform', activation = "softmax"))


        # Compile the model
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])

        
        
#         model = Sequential()
#         model.add(self.i_model)
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(128))
#         model.add(Dropout(0.1))
#         model.add(Dense(10, activation = 'softmax'))
#         model.compile(optimizer=SGD(lr=0.0001, momentum=0.99, decay=0.01), loss='categorical_crossentropy', metrics=['acc'])
        return model
    
    
    

def train(model):
        """
        Train the model with parameters:
        epochs = 5
        steps_per_epoch=100
        validation_steps=50
        
        """
   
        epochs=50
#         steps_per_epoch=10
#         validation_steps=5
        
            
        # We'll stop training if no improvement after some epochs
        earlystopper = EarlyStopping(monitor='val_acc', patience=5, verbose=1)
        reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, 
                                   verbose=1, mode='max', min_lr=0.00001)
        # Save the best model during the traning
#         checkpointer = ModelCheckpoint('best_model1.h5'
#                                         ,monitor='val_acc'
#                                         ,verbose=1
#                                         ,save_best_only=True
#                                         ,save_weights_only=True)
        # Train
        training = model.fit(x=X_train,
                             y=Y_train,
                             batch_size=32,
                             epochs=epochs,
                             verbose=1,
                             callbacks=[earlystopper, reduce_lr],
                             validation_data=(X_val, Y_val),
                             
                            )
        
        # Get the best saved weights
#         model.load_weights('best_model1.h5')
        return training

    
model = create_model()

model.summary()


In [ ]:
training=train(model)
print("Trained")



In [ ]:
f, ax = plt.subplots(1,2, figsize=(12,3))
ax[0].plot(training.history['loss'], label="Loss")
ax[0].plot(training.history['val_loss'], label="Validation loss")
ax[0].set_title('Loss')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].legend()

# Accuracy
ax[1].plot(training.history['acc'], label="Accuracy")
ax[1].plot(training.history['val_acc'], label="Validation accuracy")
ax[1].set_title('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy')
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Prediction
y_pred_raw = model.predict(X_val)
y_pred = np.argmax(y_pred_raw, axis=1)
y_true = np.argmax(Y_val, axis = 1)


In [ ]:
#Print classification report
# test_accuracy = model.evaluate(X_val)*100
print(f"Test accuracy is {np.mean(y_true == y_pred)}%")

print("Confusion Matrix")
print(confusion_matrix(y_true,y_pred ))
print('F1 score is',f1_score(y_true,y_pred, average = 'weighted') *100, "%")

predicted_probab =model.predict(X_val)
print("ROC- AUC score is", roc_auc_score( Y_val, predicted_probab)*100)


In [ ]:
def predict_for_submit():
        """
        Predict submission test data and form dataframe to submit
        """
        print("Forming submission dataframe...")
        
        # Predict
        y_pred = model.predict(test_data)
        y_pred = np.argmax(y_pred, axis=1)
       
        sample_submission.label = y_pred
    
        print(f"Submission dataframe created. Rows:{len(sample_submission.label.values)}")
        
        # Write to csv
        sample_submission.to_csv('submission.csv', index=False)
        print("Submission completed: written submission.csv")
        return sample_submission

    
sample_submission = predict_for_submit()

In [ ]:
sample_submission.head()

### References: 

https://keras.io/models/sequential/
https://keras.io/layers/core/
https://keras.io/layers/convolutional/
https://keras.io/layers/pooling/
https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6
https://www.kaggle.com/toregil/welcome-to-deep-learning-cnn-99
https://www.kaggle.com/kanncaa1/convolutional-neural-network-cnn-tutorial
https://github.com/MazenAly/Cifar100